In [ ]:
import numpy as np
import pandas as pd

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import csv
from copy import deepcopy
from skimage.measure import label
import argparse
import sys
from glob import glob
import cv2
import csv
import torch
import torch.backends.cudnn as cudnn
import yaml
from albumentations.augmentations import transforms
from albumentations.core.composition import Compose
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import KFold
from collections import OrderedDict
import pandas as pd

import torch
from torch import nn

In [ ]:
__all__ = ['UNet', 'NestedUNet']

class VGGBlock(nn.Module):
    def __init__(self, in_channels, middle_channels, out_channels):
        super().__init__()
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_channels, middle_channels, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(middle_channels)
        self.conv2 = nn.Conv2d(middle_channels, out_channels, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        return out


class UNet(nn.Module):
    def __init__(self, num_classes, input_channels=3, **kwargs):
        super().__init__()

        nb_filter = [32, 64, 128, 256, 512]

        self.pool = nn.MaxPool2d(2, 2)
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.conv0_0 = VGGBlock(input_channels, nb_filter[0], nb_filter[0])
        self.conv1_0 = VGGBlock(nb_filter[0], nb_filter[1], nb_filter[1])
        self.conv2_0 = VGGBlock(nb_filter[1], nb_filter[2], nb_filter[2])
        self.conv3_0 = VGGBlock(nb_filter[2], nb_filter[3], nb_filter[3])
        self.conv4_0 = VGGBlock(nb_filter[3], nb_filter[4], nb_filter[4])

        self.conv3_1 = VGGBlock(nb_filter[3]+nb_filter[4], nb_filter[3], nb_filter[3])
        self.conv2_2 = VGGBlock(nb_filter[2]+nb_filter[3], nb_filter[2], nb_filter[2])
        self.conv1_3 = VGGBlock(nb_filter[1]+nb_filter[2], nb_filter[1], nb_filter[1])
        self.conv0_4 = VGGBlock(nb_filter[0]+nb_filter[1], nb_filter[0], nb_filter[0])

        self.final = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)


    def forward(self, input):
        x0_0 = self.conv0_0(input)
        x1_0 = self.conv1_0(self.pool(x0_0))
        x2_0 = self.conv2_0(self.pool(x1_0))
        x3_0 = self.conv3_0(self.pool(x2_0))
        x4_0 = self.conv4_0(self.pool(x3_0))

        x3_1 = self.conv3_1(torch.cat([x3_0, self.up(x4_0)], 1))
        x2_2 = self.conv2_2(torch.cat([x2_0, self.up(x3_1)], 1))
        x1_3 = self.conv1_3(torch.cat([x1_0, self.up(x2_2)], 1))
        x0_4 = self.conv0_4(torch.cat([x0_0, self.up(x1_3)], 1))

        output = self.final(x0_4)
        return output


class NestedUNet(nn.Module):
    def __init__(self, num_classes, input_channels=3, deep_supervision=False, **kwargs):
        super().__init__()

        nb_filter = [32, 64, 128, 256, 512]

        self.deep_supervision = deep_supervision

        self.pool = nn.MaxPool2d(2, 2)
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.conv0_0 = VGGBlock(input_channels, nb_filter[0], nb_filter[0])
        self.conv1_0 = VGGBlock(nb_filter[0], nb_filter[1], nb_filter[1])
        self.conv2_0 = VGGBlock(nb_filter[1], nb_filter[2], nb_filter[2])
        self.conv3_0 = VGGBlock(nb_filter[2], nb_filter[3], nb_filter[3])
        self.conv4_0 = VGGBlock(nb_filter[3], nb_filter[4], nb_filter[4])

        self.conv0_1 = VGGBlock(nb_filter[0]+nb_filter[1], nb_filter[0], nb_filter[0])
        self.conv1_1 = VGGBlock(nb_filter[1]+nb_filter[2], nb_filter[1], nb_filter[1])
        self.conv2_1 = VGGBlock(nb_filter[2]+nb_filter[3], nb_filter[2], nb_filter[2])
        self.conv3_1 = VGGBlock(nb_filter[3]+nb_filter[4], nb_filter[3], nb_filter[3])

        self.conv0_2 = VGGBlock(nb_filter[0]*2+nb_filter[1], nb_filter[0], nb_filter[0])
        self.conv1_2 = VGGBlock(nb_filter[1]*2+nb_filter[2], nb_filter[1], nb_filter[1])
        self.conv2_2 = VGGBlock(nb_filter[2]*2+nb_filter[3], nb_filter[2], nb_filter[2])

        self.conv0_3 = VGGBlock(nb_filter[0]*3+nb_filter[1], nb_filter[0], nb_filter[0])
        self.conv1_3 = VGGBlock(nb_filter[1]*3+nb_filter[2], nb_filter[1], nb_filter[1])

        self.conv0_4 = VGGBlock(nb_filter[0]*4+nb_filter[1], nb_filter[0], nb_filter[0])

        if self.deep_supervision:
            self.final1 = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)
            self.final2 = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)
            self.final3 = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)
            self.final4 = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)
        else:
            self.final = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)


    def forward(self, input):
        x0_0 = self.conv0_0(input)
        x1_0 = self.conv1_0(self.pool(x0_0))
        x0_1 = self.conv0_1(torch.cat([x0_0, self.up(x1_0)], 1))

        x2_0 = self.conv2_0(self.pool(x1_0))
        x1_1 = self.conv1_1(torch.cat([x1_0, self.up(x2_0)], 1))
        x0_2 = self.conv0_2(torch.cat([x0_0, x0_1, self.up(x1_1)], 1))

        x3_0 = self.conv3_0(self.pool(x2_0))
        x2_1 = self.conv2_1(torch.cat([x2_0, self.up(x3_0)], 1))
        x1_2 = self.conv1_2(torch.cat([x1_0, x1_1, self.up(x2_1)], 1))
        x0_3 = self.conv0_3(torch.cat([x0_0, x0_1, x0_2, self.up(x1_2)], 1))

        x4_0 = self.conv4_0(self.pool(x3_0))
        x3_1 = self.conv3_1(torch.cat([x3_0, self.up(x4_0)], 1))
        x2_2 = self.conv2_2(torch.cat([x2_0, x2_1, self.up(x3_1)], 1))
        x1_3 = self.conv1_3(torch.cat([x1_0, x1_1, x1_2, self.up(x2_2)], 1))
        x0_4 = self.conv0_4(torch.cat([x0_0, x0_1, x0_2, x0_3, self.up(x1_3)], 1))

        if self.deep_supervision:
            output1 = self.final1(x0_1)
            output2 = self.final2(x0_2)
            output3 = self.final3(x0_3)
            output4 = self.final4(x0_4)
            return [output1, output2, output3, output4]

        else:
            output = self.final(x0_4)
            return output

In [ ]:
import zipfile
Dataset = "stage2_test_final"
os.makedirs('stage2_test', exist_ok=True)
with zipfile.ZipFile("../input/data-science-bowl-2018/"+Dataset+".zip", "r") as z:
    z.extractall("stage2_test")

img_size = 256
paths = glob('stage2_test/*')
os.makedirs('test2_256/images', exist_ok=True)
os.makedirs('test2_256/masks/0', exist_ok=True)

In [ ]:
for i in tqdm(range(len(paths))):
    path = paths[i]
    img = cv2.imread(os.path.join(path, 'images',
                     os.path.basename(path) + '.png'))
    mask = np.zeros((img.shape[0], img.shape[1]))
    if len(img.shape) == 2:
        img = np.tile(img[..., None], (1, 1, 3))
    if img.shape[2] == 4:
        img = img[..., :3]
    img = cv2.resize(img, (img_size, img_size))
    mask = cv2.resize(mask, (img_size, img_size))
    cv2.imwrite(os.path.join('test2_256/images',
                os.path.basename(path) + '.png'), img)
    cv2.imwrite(os.path.join('test2_256/masks/0',
                os.path.basename(path) + '.png'), (mask * 255).astype('uint8'))

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, img_ids, img_dir, mask_dir, img_ext, mask_ext, num_classes, transform=None):
        self.img_ids = img_ids
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.img_ext = img_ext
        self.mask_ext = mask_ext
        self.num_classes = num_classes
        self.transform = transform
    def __len__(self):
        return len(self.img_ids)
    def __getitem__(self, idx):
        img_id = self.img_ids[idx]
        
        img = cv2.imread(os.path.join(self.img_dir, img_id + self.img_ext))
        mask = []
        for i in range(self.num_classes):
            mask.append(cv2.imread(os.path.join(self.mask_dir, str(i),
                        img_id + self.mask_ext), cv2.IMREAD_GRAYSCALE)[..., None])
        mask = np.dstack(mask)
        if self.transform is not None:
            augmented = self.transform(image=img, mask=mask)
            img = augmented['image']
            mask = augmented['mask']
        
        img = img.astype('float32') / 255
        img = img.transpose(2, 0, 1)
        mask = mask.astype('float32') / 255
        mask = mask.transpose(2, 0, 1)
        
        return img, mask, {'img_id': img_id}

In [ ]:
model_path = '../input/model-sgd/Best_model/kits19_all_512_NestedUNet_woDS'

def convert_to_pixel_list(img):
    flat_img = img.ravel()
    pixel_list = []
    start_nr = 0
    run_length = 0
    for i in range(len(flat_img)):
        if flat_img[i] >= 128:
            if run_length == 0:
                start_nr = i
            run_length += 1
        else:
            if run_length > 0:
                pixel_list.append(start_nr)
                pixel_list.append(run_length)
                run_length = 0
    if run_length > 0:
        pixel_list.append(start_nr)
        pixel_list.append(run_length)
    
    result_as_string = ''
    for value in pixel_list:
        result_as_string += ' ' + str(value)
    result_as_string = result_as_string[1:] # remove first space

    return result_as_string

def generate_solution_images():
    with open(f'{model_path}/config.yml' , 'r') as f:
        config = yaml.load(f, Loader=yaml.FullLoader)

    print('-'*20)
    for key in config.keys():
        print('%s: %s' % (key, str(config[key])))
    print('-'*20)

    cudnn.benchmark = True

    # create model
    print("=> creating model %s" % config['arch'])
    model = NestedUNet(config['num_classes'], config['input_channels'], config['deep_supervision'])

    model = model.cpu()

    # Data loading code
    val_img_ids = glob('test2_256/images/*' + config['img_ext'])
    val_img_ids = [os.path.splitext(os.path.basename(p))[0] for p in val_img_ids]

    model.load_state_dict(torch.load(f'{model_path}/model.pth', map_location=torch.device('cpu')))
    model.eval()

    val_transform = Compose([
        transforms.Resize(config['input_h'], config['input_w']),
        #geometric.resize.Resize(config['input_h'], config['input_w']),
        transforms.Normalize(),
    ])

    val_dataset = Dataset(
        img_ids=val_img_ids,
        img_dir='test2_256/images/',
        mask_dir='test2_256/masks/',
        img_ext=config['img_ext'],
        mask_ext=config['mask_ext'],
        num_classes=config['num_classes'],
        transform=val_transform)
    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=config['batch_size'],
        shuffle=False,
        num_workers=config['num_workers'],
        drop_last=False)

    for c in range(config['num_classes']):
        os.makedirs(os.path.join(f'outputs', str(c)), exist_ok=True)
    
    with torch.no_grad():
        for input, target, meta in tqdm(val_loader, total=len(val_loader)):
            input = input.cpu()
            target = target.cpu()

            # compute output
            if config['deep_supervision']:
                output = model(input)[-1]
            else:
                output = model(input)

            output = torch.sigmoid(output).cpu().numpy()
            for i in range(len(output)):
                for c in range(config['num_classes']):
                    out_img = (output[i, c] * 255).astype('uint8')
                    id = meta['img_id'][i]
                    cv2.imwrite(os.path.join(f'outputs', str(c), meta['img_id'][i] + '.jpg'), out_img)
    #torch.cuda.empty_cache()

generate_solution_images()

In [ ]:
def rle_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    result_as_string = ''
    for value in run_lengths:
        result_as_string += ' ' + str(value)
    result_as_string = result_as_string[1:] # remove first space

    return result_as_string

def all_pixel_lists(images):
    results = []
    for img in images:
        results.append(rle_encoding(img))
    return results

def segmented_images(mask):
    results = []
    lab_img = label(mask)
    for i in range(1, lab_img.max() + 1):
        img = lab_img == i
        if np.count_nonzero(img) > 5:
            results.append(img)
    return results

def main():
    id_to_shape = {}
    original_img_dir = glob(os.path.join('stage2_test', "*"))
    for img_dir in original_img_dir:
        img_id = os.path.basename(os.path.normpath(img_dir)) 
        image_path = os.path.join(img_dir, 'images', img_id+'.png')
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        id_to_shape[img_id] = (img.shape[1], img.shape[0])

    with open(f'test.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['ImageId', 'EncodedPixels'])
        img_paths = glob(os.path.join(f'outputs/0/', "*"))
        k = 0
        avg_scores = []
        for img_path in img_paths:
            print(f'\r{k+1}/{len(img_paths)}', end='')
            k += 1
            img_id = os.path.basename(os.path.normpath(img_path))[:-4]
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, id_to_shape[img_id])
            _, img = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY)
            seg_imgs = segmented_images(img)
            pixel_lists = all_pixel_lists(seg_imgs)
            for pixel_list in pixel_lists:
                writer.writerow([img_id, pixel_list])
            if len(pixel_list) == 0:
                writer.writerow([img_id, ''])

main()